<a href="https://colab.research.google.com/github/HusseinAndia/Mask-Detection/blob/master/Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.applications.mobilenet import MobileNet
from keras.models import Model, Sequential
from keras.layers import GlobalAveragePooling2D, Dropout, Dense, Activation, BatchNormalization, Conv2D, MaxPool2D, Flatten, MaxPooling2D
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import SGD, Adam
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import random
import os


Using TensorFlow backend.


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# **Parameters**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! cp "/content/gdrive/My Drive/Projects/MaskDetection/faces1.zip" .

In [ ]:
! unzip faces1.zip

Archive:  faces1.zip
   creating: faces1/maskoff/
  inflating: faces1/maskoff/maskoff (185).jpg  
  inflating: faces1/maskoff/maskoff (186).jpg  
  inflating: faces1/maskoff/maskoff (187).jpg  
  inflating: faces1/maskoff/maskoff (188).jpg  
  inflating: faces1/maskoff/maskoff (189).jpg  
  inflating: faces1/maskoff/maskoff (190).jpg  
  inflating: faces1/maskoff/maskoff (191).jpg  
  inflating: faces1/maskoff/maskoff (192).jpg  
  inflating: faces1/maskoff/maskoff (193).jpg  
  inflating: faces1/maskoff/maskoff (194).jpg  
  inflating: faces1/maskoff/maskoff (195).jpg  
  inflating: faces1/maskoff/maskoff (196).jpg  
  inflating: faces1/maskoff/maskoff (197).jpg  
  inflating: faces1/maskoff/maskoff (198).jpg  
  inflating: faces1/maskoff/maskoff (199).jpg  
  inflating: faces1/maskoff/maskoff (200).jpg  
  inflating: faces1/maskoff/maskoff (201).jpg  
  inflating: faces1/maskoff/maskoff (202).jpg  
  inflating: faces1/maskoff/maskoff (203).jpg  
  inflating: faces1/maskoff/maskoff (2

In [ ]:
MODEL_SAVE = 'service_type_model.h5'
FAST_RUN = False
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 1
IMG_DIR = 'faces1/'
BATCH_SIZE = 32
NUM_CLASSES = 2

# **Build Model**

In [ ]:
dataset = []

for fold in os.listdir(IMG_DIR):
    for filename in os.listdir(f'{IMG_DIR}/{fold}'):
        dataset.append((f'{fold}/{filename}', fold))

df = pd.DataFrame(dataset, columns=['filename', 'category'])
df_train, df_test = train_test_split(df, random_state=42, stratify=df.category, test_size=.2)
df_train['set'] = 'train'
df_test['set'] = 'test'
df = df_train.append(df_test)
df.to_csv('dataset.csv', index=False)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,filename,category,set
1817,maskoff/maskoff (204).jpg,maskoff,train
439,maskon/augmented_image_216.jpg,maskon,train
35,maskon/novissimas (160).jpg,maskon,train
1137,maskoff/unknown (933).jpg,maskoff,train
702,maskon/novissimas (216).jpg,maskon,train


In [ ]:
df = pd.read_csv('dataset.csv')
df.head()

,filename,category,set
0,maskoff/maskoff (204).jpg,maskoff,train
1,maskon/augmented_image_216.jpg,maskon,train
2,maskon/novissimas (160).jpg,maskon,train
3,maskoff/unknown (933).jpg,maskoff,train
4,maskon/novissimas (216).jpg,maskon,train


In [ ]:
train_df = df[df.set == 'train'].reset_index(drop=True)
validate_df = df[df.set == 'test'].reset_index(drop=True)

In [ ]:
base_model = MobileNet(
    weights= None, 
    include_top=False, 
    input_shape= (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256,activation='relu')(x) 
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

opt = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
callbacks_list = [
    ModelCheckpoint('model-{epoch:03d}.model', monitor='val_accuracy', verbose=1, 
                    save_best_only=True, mode='max'),
    EarlyStopping(monitor='val_accuracy', patience=12),
    ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
]

In [ ]:
def add_noise(img):
    '''Add random noise to an image'''
    VARIABILITY = 8
    deviation = VARIABILITY*random.random()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img


train_datagen = ImageDataGenerator(
    brightness_range=[0.2, 1.6],
    rescale=1. / 255,
    rotation_range=0, 
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode="nearest",
    preprocessing_function=add_noise,
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    IMG_DIR, 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    color_mode = 'grayscale',
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

Found 1498 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 4 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    IMG_DIR,
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    color_mode = 'grayscale',
    class_mode='categorical',
    shuffle=False,
    batch_size=BATCH_SIZE
)

Found 374 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


## Fit model

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

In [ ]:
epochs=50 if FAST_RUN else 50

history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//BATCH_SIZE,
    steps_per_epoch=total_train//BATCH_SIZE,
    callbacks=callbacks_list
)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/50
46/46 [==============================] - 25s 542ms/step - loss: 0.7443 - accuracy: 0.5580 - val_loss: 0.7001 - val_accuracy: 0.5000

Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to model-001.model
Epoch 2/50
46/46 [==============================] - 20s 437ms/step - loss: 0.6277 - accuracy: 0.6453 - val_loss: 0.6921 - val_accuracy: 0.4942

Epoch 00002: val_accuracy did not improve from 0.50000
Epoch 3/50
46/46 [==============================] - 20s 432ms/step - loss: 0.5572 - accuracy: 0.7128 - val_loss: 0.6978 - val_accuracy: 0.4883

Epoch 00003: val_accuracy did not improve from 0.50000
Epoch 4/50
46/46 [==============================] - 20s 428ms/step - loss: 0.5003 - accuracy: 0.7558 - val_loss: 0.6976 - val_accuracy: 0.4971

Epoch 00004: val_accuracy did not improve from 0.50000

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.000500000023748